In [ ]:
# make sure CUDA is installed
!nvcc --version

# make sure you have a GPU runtime (if this fails go to runtime -> change runtime type)
!nvidia-smi

# Install some magic to run and save .cpp programs
!curl -o ./cpu_runner.py https://raw.githubusercontent.com/COMS-BC3159-F24/helpers/main/cpu_runner.py
%load_ext cpu_runner

# Install some magic to run and save .cu C++ CUDA programs
!curl -o ./gpu_runner.py https://raw.githubusercontent.com/COMS-BC3159-F24/helpers/main/gpu_runner.py
%load_ext gpu_runner

# to learn about how to do more fancy things with CUDA using this API see:
# https://nvcc4jupyter.readthedocs.io/en/latest/index.html

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
Tue Dec 10 22:35:26 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8               9W /  7

In [ ]:
!curl -o ./matrix_lib.o https://raw.githubusercontent.com/COMS-BC3159-F24/helpers/main/matrix_lib.o
!curl -o ./matrix_lib.h https://raw.githubusercontent.com/COMS-BC3159-F24/helpers/main/matrix_lib.h

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2336  100  2336    0     0   4831      0 --:--:-- --:--:-- --:--:--  4826
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   234  100   234    0     0    643      0 --:--:-- --:--:-- --:--:--   644


This is multithreaded numpy code for solving a linear system of equation. It runs faster than true serial code for solving a linear system of equation.
It uses multiprocessing library to support using threads across multiple processors (so parallel processing + multithreading; do note that Google Colab supports 2 vCPU processors)

In [ ]:
import numpy as np
from concurrent.futures import ThreadPoolExecutor
import time
import os
from multiprocessing import Pool

def row_multiply(args):
    row, matrix2 = args
    return np.dot(row, matrix2)

def multithreaded_matrix_multiply(matrix1, matrix2):
    with Pool() as pool:
        results = pool.map(row_multiply, [(row, matrix2) for row in matrix1])
    return np.array(results)


def invert_matrix(matrix):
    """Invert a single matrix."""
    return np.linalg.inv(matrix)

def multithreaded_matrix_inversion(matrices):
    """
    Perform matrix inversion on a list of matrices using multithreading.
    """
    with ThreadPoolExecutor(max_workers=os.cpu_count()*2) as executor:
        results = list(executor.map(invert_matrix, matrices))

    return results

# Example usage
if __name__ == "__main__":
    # Random matrices for demonstration
    matrix_A = np.random.randint(1, 11, size=(100, 100))
    matrix_B = np.random.randint(1, 11, size=(100, 1))

    # print("Matrix A:")
    # print(matrix_A)
    # print()

    # print("Matrix B:")
    # print(matrix_B)
    # print()

    # Multithreaded matrix inversion
    matrices_to_invert = [matrix_A]  # Can add more matrices to this list

    start = time.perf_counter()
    inverted_matrices = multithreaded_matrix_inversion(matrices_to_invert)

    # print("Inverted Matrix A:")
    # print(inverted_matrices[0])

    matrix_X = multithreaded_matrix_multiply(inverted_matrices[0], matrix_B)

    end = time.perf_counter()
    elapsed = end - start
    print(f"Elapsed time: {elapsed:.6f} seconds")

    # print("Matrix X:")
    # print(matrix_X)
    # print("Matrix X:")
    # print(matrix_X_serial)


Elapsed time: 0.059147 seconds


This is true serial way of solving a system of linear equations by hand (no huge optimizations or innate multithreading that numpy does for you). It runs slower than the multithreaded numpy code

In [ ]:
import time
import numpy as np

def matrix_inverse(matrix):
    # Get the number of rows/columns
    n = len(matrix)

    # Create augmented matrix (matrix | identity)
    augmented_matrix = [row[:] + [1 if i == j else 0 for j in range(n)] for i, row in enumerate(matrix)]

    # Perform Gaussian elimination
    for i in range(n):
        # Make the diagonal element 1
        if augmented_matrix[i][i] == 0:
            raise ValueError("Matrix is singular and cannot be inverted.")

        # Scale the row so that the pivot element is 1
        scale = augmented_matrix[i][i]
        for j in range(2 * n):
            augmented_matrix[i][j] /= scale

        # Eliminate the column below the pivot
        for j in range(i + 1, n):
            factor = augmented_matrix[j][i]
            for k in range(2 * n):
                augmented_matrix[j][k] -= factor * augmented_matrix[i][k]

    # Back substitution
    for i in range(n - 1, -1, -1):
        for j in range(i - 1, -1, -1):
            factor = augmented_matrix[j][i]
            for k in range(2 * n):
                augmented_matrix[j][k] -= factor * augmented_matrix[i][k]

    # Extract the inverse matrix from the augmented matrix
    inverse_matrix = [row[n:] for row in augmented_matrix]
    return inverse_matrix

def matrix_multiply(A, B):
    # Check if multiplication is possible (columns of A == rows of B)
    if len(A[0]) != len(B):
        raise ValueError("Matrix dimensions are incompatible for multiplication.")

    # Initialize result matrix with zeros
    result = [[0] * len(B[0]) for _ in range(len(A))]

    for i in range(len(A)):  # Iterate over rows of A
        for j in range(len(B[0])):  # Iterate over columns of B
            for k in range(len(B)):  # Iterate over rows of B
                result[i][j] += A[i][k] * B[k][j]
    return result


# Example usage
if __name__ == "__main__":
    # Random matrices for demonstration
    matrix_A = np.random.randint(1, 11, size=(100, 100)).tolist()  # Convert numpy array to list of lists
    matrix_B = np.random.randint(1, 11, size=(100, 1)).tolist()    # Convert numpy array to list of lists

    # Invert matrix A
    start = time.perf_counter()
    inverted_matrix_A = matrix_inverse(matrix_A)  # Pass matrix_A instead of matrices_to_invert

    # Multiply inverted matrix A by matrix B
    matrix_X_serial = matrix_multiply(inverted_matrix_A, matrix_B)

    end = time.perf_counter()
    elapsed = end - start
    print(f"Elapsed time: {elapsed:.6f} seconds")

Elapsed time: 0.370257 seconds


This is the serial way of solving a system of linear equations using numpy, but because numpy is automatically multithreaded (since it uses OpenBLAS or MKL, which the environment variable for these on the OS may prefer more than 1 thread) this technically is multithreaded...

In [ ]:
import numpy as np
from concurrent.futures import ThreadPoolExecutor
import time

def matrix_multiply_serial(matrix1, matrix2):
    """
    Perform matrix multiplication in a single-threaded/serial manner.
    """
    if matrix1.shape[1] != matrix2.shape[0]:
        raise ValueError("Matrix dimensions are incompatible for multiplication.")

    result = np.zeros((matrix1.shape[0], matrix2.shape[1]))
    for i in range(matrix1.shape[0]):  # Iterate over rows of matrix1
        for j in range(matrix2.shape[1]):  # Iterate over columns of matrix2
            result[i, j] = np.dot(matrix1[i, :], matrix2[:, j])  # Compute dot product
    return result

def invert_matrix_serial(matrix):
    """Invert a single matrix."""
    return np.linalg.inv(matrix)

def invert_matrices_serial(matrices):
    """
    Invert a list of matrices in a single-threaded/serial manner.
    """
    results = [invert_matrix_serial(matrix) for matrix in matrices]
    return results

# Example usage
if __name__ == "__main__":
    # Random matrices for demonstration
    matrix_A = np.random.randint(1, 11, size=(5000, 5000))
    matrix_B = np.random.randint(1, 11, size=(5000, 1))

    # print("Matrix A:")
    # print(matrix_A)
    # print()

    # print("Matrix B:")
    # print(matrix_B)
    # print()

    # Single-threaded matrix inversion
    matrices_to_invert = [matrix_A]  # Can add more matrices to this list

    start = time.perf_counter()
    inverted_matrices = invert_matrices_serial(matrices_to_invert)
    matrix_X_serial = matrix_multiply_serial(inverted_matrices[0], matrix_B)

    end = time.perf_counter()
    elapsed = end - start
    print(f"Elapsed time: {elapsed:.6f} seconds")

    # print("Matrix X:")
    # print(matrix_X_serial)


Elapsed time: 15.528819 seconds


In [ ]:
import os

print(os.cpu_count()) # Only 2 cores, means 2 threads are being used in multithreading...

2


In [ ]:
%%gpurun -n my_kernel.cu
#include <cstdio>
#include <cuda_runtime.h>

// This function runs on the GPU
__global__ void helloWorldKernel() {
    printf("Hello World!\n");
}

// This function runs on the CPU
__host__
int main() {
    // Launch the kernel on the GPU
    helloWorldKernel<<<1, 1>>>();

    // Wait for the kernel to finish executing
    cudaDeviceSynchronize();

    return 0;
}


Hello World!



In [ ]:
%%gpurun -n cuda_matmul.cu -o matrix_lib.o
#include <iostream>
#include <cmath>
#include <cuda_runtime.h>
#include "matrix_lib.h"
#define THREAD_BLOCK_SIZE 6  // Number of threads per block in each dimension

// Paste your Problem 3 solution for comparison
void matrixMultiplyCPU_float(float* A, float* B, float* C, int M, int N, int P) {
  for (int i = 0; i <  M; i++){
    for (int j = 0; j < P; j++){
        float sum = 0;
        for (int k = 0; k < N; k++) {
            sum += A[k + i * N] * B[j + k * P];
        }
        C[j + i * P] = sum;
    }
  }
}

/*
// CUDA kernel to multiply two matrices A[M][N] * B[N][P] = C[M][P]
// Specify what work to complete via threadIdx, blockIdx, blockDim
// Compute one element of the resulting matrix
__global__ void matrixMultiplyCUDA(float* A, float* B, float* C, int M, int N, int P) {

    for (int j = threadIdx.y; j < M; j += blockDim.y) {
      for (int i = threadIdx.x; i < P; i += blockDim.x){
          // every thread is adding to this sum value because they share this
          // sum variable (because they have shared memory)
          float sum = 0;
          for (int k = 0; k < N; k++) {
            sum += A[j * N + k] * B[k * P + i];
          }
          C[j * P + i] = sum;
      }
      __syncthreads();
    }
}
*/

__global__ void matrixMultiplyCUDA(float* A, float* B, float* C, int M, int N, int P) {
    int tx = threadIdx.x;  // Local thread ID in x
    int ty = threadIdx.y;  // Local thread ID in y

    int blockRow = blockIdx.y * THREAD_BLOCK_SIZE;
    int blockCol = blockIdx.x * THREAD_BLOCK_SIZE;

    // Each thread computes a tile of the output matrix
    for (int row = blockRow + ty; row < blockRow + THREAD_BLOCK_SIZE && row < N; row += THREAD_BLOCK_SIZE) {
        for (int col = blockCol + tx; col < blockCol + THREAD_BLOCK_SIZE && col < N; col += THREAD_BLOCK_SIZE) {
            float value = 0.0f;

            // Perform the multiplication for this element
            for (int k = 0; k < N; ++k) {
                value += A[row * N + k] * B[k * N + col];
            }

            // Write the result to the output matrix, with bounds check
            if (row < N && col < N) {
                C[row * N + col] = value;
            }
        }
    }
}


// GPU Device Function
// - actually solve the matrix inverse!
__device__
void matrix_inverse_inner(float *s_input, float *s_output, float *s_temp, const int matrix_dim){
  // Set up the matrix with identity next to it
  for (int i = 0, j = 0; i < matrix_dim; i++) {
      s_temp[i * matrix_dim + j] = 1;
      j++;
  }
  // all threads needs to wait on s_temp being filled appropriately
  __syncthreads();

  // Do Guassian elimination walking down the matrix (assuming no leading 0s).
  // We therefore use the columns in order as the pivot column for each pivot we need to rescale
  // that row so that the pivot value is 1 THEN for all other row values we need to add a multiple
  // of the NEW pivot row value such that we transorm the other row pivot column value to 0.
  // See https://www.mathsisfun.com/algebra/matrix-inverse-row-operations-gauss-jordan.html
  //
  // Note if you would prefer to use another method that is fine but/and this is the method
  // we have a solution for and are prepared to help you with!

  // consider threads equaling matrix dims
  for (unsigned pivRC = 0, j = 0; pivRC < matrix_dim; pivRC++){
      float leading_rc = s_input[pivRC * matrix_dim + j];

      // first divide in the pivot row with the leading_rc
      s_temp[pivRC * matrix_dim + threadIdx.x] /= leading_rc;
      s_input[pivRC * matrix_dim + threadIdx.x] /= leading_rc;

      // wait for all threads to perform division on the pivot row
      __syncthreads();

      // each thread will take control of a row in a column and perform subtraction on
      // that row.
      // outer for loop is used in the event that there's less threads used than there
      // are with matrix_dim
      for (unsigned i = threadIdx.x; i < matrix_dim; i += blockDim.x) {
        unsigned mult_by_ind = i * matrix_dim + j;
        unsigned row_start = i * matrix_dim;
        // don't perform subtraction on pivot row
        if (mult_by_ind != (pivRC * matrix_dim + j)) {
            float mult_by = s_input[mult_by_ind];
            for (int col = 0; col < matrix_dim; col += 1) {
                // subtract the identity matrix appropriately
                s_temp[row_start + col] -=  mult_by * s_temp[pivRC * matrix_dim + col];
                // printf("%f\n", s_temp[row_start + col]);
                // subtract the input matrix appropriately
                s_input[row_start + col] -= mult_by * s_input[pivRC * matrix_dim + col];
            }
        }
      }
      // wait for all threads to perform subtraction on all other rows besides
      // pivot row to ready up for next iteration
      __syncthreads();


      j++;
  }
  // make sure all threads are done with their work in the for loop
  // before writing output
  __syncthreads();

  // Make sure to write the result to the output
  for (unsigned i = threadIdx.x; i < matrix_dim * matrix_dim; i += blockDim.x) {
      s_output[i] = s_temp[i];
  }
  __syncthreads();
}


// GPU kernel
// - Set up shared memory, run the _inner, clean up shared memory
__global__
void matrix_inverse_kernel(float *d_input, float *d_output, const int matrix_dim){
  // get shared pointers
  extern __shared__ float s_dynShared[];
  float *s_input = (float *) (s_dynShared);
  float *s_output = (float *) (s_dynShared + (matrix_dim * matrix_dim) * sizeof(float));
  float *s_temp = (float *) (s_dynShared + 2 * (matrix_dim * matrix_dim) * sizeof(float));

  // printf("%f\n", s_input[0]);
  // copy the d_input data into shared memory
  // do this a bit faster through threads
  for (int i = threadIdx.x; i < matrix_dim * matrix_dim; i += blockDim.x) {
      s_input[i] = d_input[i];
      s_temp[i] = 0;
  }

  // wait for all threads to finish populating the arrays
  __syncthreads();

  // printf("%f\n", s_input[0]);
  // run the code

  matrix_inverse_inner(s_input, s_output, s_temp, matrix_dim);
  // wait for this matrix_inverse_inner function to finish
  // actually no need to sync here because the function does it for us
  // __syncthreads();

  // copy the memory back out to d_output

  for (int i = threadIdx.x; i < matrix_dim * matrix_dim; i += blockDim.x) {
    d_output[i] = s_output[i];
  }
  // make sure the output to give to host is all synced up.
  __syncthreads();
}


// Host function
// - set up GPU memory, run the kernel, clean up GPU memory
__host__
void matrix_inverse(float *h_input, float *h_output, const int matrix_dim){

  // transfer memory to the device
  // float* h_in = (float *) malloc(matrix_dim * matrix_dim * sizeof(float));
  // float* h_out = (float *) malloc(matrix_dim * matrix_dim * sizeof(float));
  float *d_input, *d_output;
  cudaMalloc(&d_input, matrix_dim * matrix_dim * sizeof(float));
  cudaMalloc(&d_output, matrix_dim * matrix_dim * sizeof(float));
  cudaMemcpy(d_input, h_input, matrix_dim * matrix_dim * sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_output, h_output, matrix_dim * matrix_dim * sizeof(float), cudaMemcpyHostToDevice);

  // printf("%f\n", h_input[0]);
  // run the kernel
  matrix_inverse_kernel<<<1, matrix_dim, 3 * (matrix_dim * matrix_dim * sizeof(float) * sizeof(float *))>>>(d_input, d_output, matrix_dim);
  cudaDeviceSynchronize();

  // transfer data back to the host and clean up
  cudaMemcpy(h_output, d_output, matrix_dim * matrix_dim * sizeof(float), cudaMemcpyDeviceToHost);
  cudaFree(d_input);
  cudaFree(d_output);
}


// Compare matrices represented as flat arrays with single loop
// Allow for some floating-point error (GPU and CPU may differ slightly)
bool compareMatrices(const float* A, const float* B, int total_elements) {
    float epsilon = 0.001f; // Tolerance for floating-point comparison
    int precision_issues = 0;

    // Note: not traversing in row-major order
    for (int i = 0; i < total_elements; ++i) {
        float diff = fabs(A[i] - B[i]);
        if (diff > epsilon) {
            printf("Matrices differ at element %d: A[%d] = %f, B[%d] = %f\n", i, i, A[i], i, B[i]);
            return false;
        } else if (diff > epsilon / 10 && diff <= epsilon) {
            precision_issues++;
        }
    }

    if (precision_issues > 0) {
        printf("Warning: %d elements had values close to the precision threshold.\n", precision_issues);
    }

    return true;
}

int main() {
    const int rows_A = 5, cols_A = 5;
    const int rows_B = 5, cols_B = 5;
    const int rows_C = rows_A, cols_C = cols_B;

    const int matrix_dim = 5;
    const int matrix_dim_sq = matrix_dim*matrix_dim;

    float *h_input = (float *) malloc(matrix_dim_sq*sizeof(float));

    // Fill matrix A and B (similar to as you did in the CPU version)
    float A_arr[rows_A * cols_A] = {
        2, 1, 3, 0, -1,
        0, 1, 2, 4, 1,
        1, 0, 1, 2, 3,
        4, -2, 1, 3, 0,
        3, 2, -1, 1, 1
    };

    for (int i = 0; i < rows_A * cols_A; i++) {
        h_input[i] = A_arr[i];
    }
    float *h_output = (float *)malloc(matrix_dim_sq*sizeof(float));

    // Allocate memory for matrices A, B, C
    // * C_gpu will be for copying and comparison purposes
    float* h_A = (float *) malloc(rows_A * cols_A * sizeof(float)); // TODO
    float* h_B = (float *) malloc(rows_B * cols_B * sizeof(float)); // TODO
    float* h_C_cpu = (float *) malloc(rows_C * cols_C * sizeof(float)); // TODO
    float* h_C_gpu = (float *) malloc(rows_C * cols_C * sizeof(float));


    if (h_A == nullptr) {
        fprintf(stderr, "Memory allocation for h_A failed\n");
        return 1;
    }

    if (h_B == nullptr) {
        fprintf(stderr, "Memory allocation for h_B failed\n");
        return 1;
    }

    if (h_C_cpu == nullptr) {
        fprintf(stderr, "Memory allocation for h_C_cpu failed\n");
        return 1;
    }

    if (h_C_gpu == nullptr) {
        fprintf(stderr, "Memory allocation for h_C_gpu failed\n");
        return 1;
    }

    matrix_inverse(h_input,h_output,matrix_dim);

    print_float_matrix(h_output, matrix_dim, matrix_dim);
    for (int i = 0; i < rows_A * cols_A; i++) {
        h_A[i] = h_output[i];
    }

    float B_arr[rows_B * cols_B] = {
      20.0, 21.0, 22.0, 23.0,
      24.0, 25.0, 26.0, 27.0,
      28.0, 29.0, 30.0, 31.0,
      32.0, 33.0, 34.0, 35.0,
      36.0, 37.0, 38.0, 39.0,
      40.0, 41.0, 42.0, 43.0,
      44.0
    };

    for (int i = 0; i < rows_B * cols_B; i++) {
        h_B[i] = B_arr[i];
    }

    float C_arr[cols_C * rows_C] = {0.0};

    for (int i = 0; i < rows_C * cols_C; i++) {
        h_C_cpu[i] = C_arr[i];
        h_C_gpu[i] = C_arr[i];
    }

    // Perform matrix multiplication on CPU (C++)
    matrixMultiplyCPU_float(h_A, h_B, h_C_cpu, rows_A, cols_A, cols_B);
    printf("CPU Matrix Multiply:\n");
    print_float_matrix(h_C_cpu, rows_C, cols_C);

    // Perform matrix multiplication on GPU (CUDA)
    // create d_ device pointers, allocate GPU memory, and fill the memory
    float *d_A, *d_B, *d_C;
    cudaMalloc(&d_A, rows_A * cols_A * sizeof(float));
    cudaMalloc(&d_B, rows_B * cols_B * sizeof(float));
    cudaMalloc(&d_C, rows_C * cols_C * sizeof(float));
    cudaMemcpy(d_A, h_A, rows_A * cols_A * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, rows_B * cols_B * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_C, h_C_gpu, rows_C * cols_C * sizeof(float), cudaMemcpyHostToDevice);

    // dim3 grid(1, 1);
    // dim3 block(2, 2);
    dim3 block(THREAD_BLOCK_SIZE, THREAD_BLOCK_SIZE);
    dim3 grid((cols_A + THREAD_BLOCK_SIZE - 1) / THREAD_BLOCK_SIZE, (cols_A + THREAD_BLOCK_SIZE - 1) / THREAD_BLOCK_SIZE);

    // Define grid/block dimensions, then launch the kernel
    matrixMultiplyCUDA<<<grid, block>>>(d_A, d_B, d_C, rows_A, cols_A, cols_B);
    cudaDeviceSynchronize();

    // Copy result back to host (h_C_gpu) for comparison
    cudaMemcpy(h_C_gpu, d_C, rows_C * cols_C * sizeof(float), cudaMemcpyDeviceToHost);

    printf("\nGPU Matrix Multiply:\n");
    print_float_matrix(h_C_gpu, rows_C, cols_B);

    // Compare the CPU and GPU results
    if (compareMatrices(h_C_cpu, h_C_gpu, rows_C*cols_C)) {
        printf("\nMatrices match!\n");
    } else {
        printf("\nMatrices do NOT match.\n");
    }

    // Free memory
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);
    free(h_A);
    free(h_B);
    free(h_C_cpu);
    free(h_C_gpu);
    free(h_input);

    return 0;
}


   0.08   -0.12    0.03    0.11    0.13 
   0.08    0.13   -0.09   -0.18    0.22 
   0.25   -0.00    0.14   -0.04   -0.16 
  -0.14    0.25   -0.14    0.08    0.03 
  -0.02   -0.13    0.37   -0.08   -0.01 
CPU Matrix Multiply:
   8.15    8.37    8.58    8.80    9.02 
   4.44    4.60    4.75    4.90    5.06 
   1.30    1.48    1.67    1.85    2.04 
   3.33    3.42    3.50    3.58    3.67 
   4.63    4.77    4.92    5.06    5.20 

GPU Matrix Multiply:
   8.15    8.37    8.58    8.80    9.02 
   4.44    4.60    4.75    4.90    5.06 
   1.30    1.48    1.67    1.85    2.04 
   3.33    3.42    3.50    3.58    3.67 
   4.63    4.77    4.92    5.06    5.20 

Matrices match!

